# Predicción de MP2.5 con LSTM y features enriquecidas

In [ ]:

import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
import matplotlib.pyplot as plt


## Carga de datos

In [ ]:

def convertir_fechas(df):
    df['Timestamp_Pronostico'] = pd.to_datetime(df['Timestamp_Pronostico'])
    df['Fecha'] = pd.to_datetime(df['Fecha']).dt.date
    return df

pronostico_met_df = pd.concat([
    pd.read_csv('pronostico_consolidado_2022_Curico.csv'),
    pd.read_csv('pronostico_consolidado_2023_Curico.csv'),
    pd.read_csv('pronostico_consolidado_2024_Curico.csv')
])

pronostico_co_df = pd.concat([
    pd.read_csv('pronostico_consolidado_completo_CO_2022.csv'),
    pd.read_csv('pronostico_consolidado_completo_CO_2023.csv'),
    pd.read_csv('pronostico_consolidado_completo_CO_2024.csv')
])

pronostico_met_df = convertir_fechas(pronostico_met_df)
pronostico_co_df = convertir_fechas(pronostico_co_df)

df_pronosticos = pd.merge(
    pronostico_met_df,
    pronostico_co_df,
    on=['Fecha', 'Timestamp_Pronostico', 'Region'],
    how='inner'
)


In [ ]:

def limpiar_pronosticos(df, need_cut=True, first_val=-1):
    df = df.sort_values(by="Timestamp_Pronostico")
    if need_cut:
        df = df[4:-3]
    df['horizonte'] = [i for i in range(first_val, 4) for _ in range(24)]
    return df

df_pronosticos = df_pronosticos.groupby('Fecha').apply(limpiar_pronosticos).reset_index(drop=True)


## Carga de observados y cálculo de media móvil

In [ ]:

observados_df = pd.read_csv('datos_observados_curico.csv', index_col=0)[[
    'Timestamp', 'TEMP', 'WDIR', 'RHUM', 'WSPD', 'MP25_validado', 'MP25_preliminar', 'MP25_no_validado']]
observados_df['Timestamp'] = pd.to_datetime(observados_df['Timestamp'])

def mm(df, columna, decimals=0, ventana=24):
    def promedio_personalizado(x):
        no_nulos = x.dropna()
        if len(x) - len(no_nulos) < 7:
            return round(no_nulos.mean(), decimals)
        else:
            return None
    return df[columna].rolling(window=ventana, min_periods=1).apply(promedio_personalizado, raw=False)

observados_df = observados_df.sort_values('Timestamp')
observados_df['MP25_mm'] = mm(observados_df, 'MP25_validado', decimals=2)


## Generación de features compuestas y temporales

In [ ]:

df_pronosticos['TEMP_WSPD'] = df_pronosticos['TEMP'] * df_pronosticos['WSPD']
df_pronosticos['CO_TEMP'] = df_pronosticos['CO(CENTRO)'] * df_pronosticos['TEMP']
df_pronosticos['CO_SO2'] = df_pronosticos['CO(CENTRO)'] / (df_pronosticos['SO2(CENTRO)'] + 1e-5)

df_pronosticos['hour'] = df_pronosticos['Timestamp_Pronostico'].dt.hour
df_pronosticos['dayofweek'] = df_pronosticos['Timestamp_Pronostico'].dt.dayofweek
df_pronosticos['dayofyear'] = df_pronosticos['Timestamp_Pronostico'].dt.dayofyear

df_pronosticos['hour_sin'] = np.sin(2 * np.pi * df_pronosticos['hour'] / 24)
df_pronosticos['hour_cos'] = np.cos(2 * np.pi * df_pronosticos['hour'] / 24)
df_pronosticos['dayofweek_sin'] = np.sin(2 * np.pi * df_pronosticos['dayofweek'] / 7)
df_pronosticos['dayofweek_cos'] = np.cos(2 * np.pi * df_pronosticos['dayofweek'] / 7)
df_pronosticos['dayofyear_sin'] = np.sin(2 * np.pi * df_pronosticos['dayofyear'] / 365)
df_pronosticos['dayofyear_cos'] = np.cos(2 * np.pi * df_pronosticos['dayofyear'] / 365)

df_pronosticos['WDIR_sin'] = np.sin(np.deg2rad(df_pronosticos['WDIR']))
df_pronosticos['WDIR_cos'] = np.cos(np.deg2rad(df_pronosticos['WDIR']))

feature_cols = [
    'TEMP', 'WSPD', 'WDIR_sin', 'WDIR_cos', 'CO(CENTRO)', 'SO2(CENTRO)',
    'hour_sin', 'hour_cos', 'dayofweek_sin', 'dayofweek_cos', 'dayofyear_sin', 'dayofyear_cos',
    'TEMP_WSPD', 'CO_TEMP', 'CO_SO2'
]
